# Categorização Manual dos Clusters

Este notebook guia você através do processo de categorização manual dos clusters identificados pelo modelo não supervisionado. O objetivo é atribuir significado semântico aos padrões descobertos automaticamente.

## Processo de Categorização:
1. Visualizar amostras de cada cluster
2. Identificar padrões visuais
3. Atribuir categorias significativas
4. Documentar o processo de decisão
5. Salvar as categorizações para uso futuro

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from src.models.cluster_categorizer import ClusterCategorizer
from src.models.unsupervised_cnn import UnsupervisedCNN
from src.data.tiff_loader import TiffLoader

%matplotlib inline
plt.style.use('seaborn')

## 1. Carregar o Modelo e os Dados

In [ ]:
# Carregar o modelo treinado (ajuste o caminho conforme necessário)
model = UnsupervisedCNN.load_model('path_to_your_model')

# Carregar alguns dados para visualização
loader = TiffLoader(train_val_dir='../data/train')
train_ds, _ = loader.load_train_val_data()
X_samples = np.concatenate([batch.numpy() for batch in train_ds.take(10)], axis=0)
predictions = model.predict(X_samples)

## 2. Visualizar Clusters para Categorização

Vamos criar um categorizador e visualizar amostras de cada cluster:

In [ ]:
# Criar categorizador
categorizer = ClusterCategorizer(n_clusters=model.n_clusters)

# Visualizar amostras de cada cluster
categorizer.plot_cluster_samples(X_samples, predictions)

## 3. Categorização Manual

Agora vamos categorizar cada cluster baseado nas características observadas. Algumas categorias comuns em sensoriamento remoto incluem:

- Vegetação densa (floresta)
- Vegetação esparsa
- Área urbana
- Solo exposto
- Corpo d'água
- Área agrícola
- Pasto

Para cada cluster, vamos:
1. Observar as características visuais
2. Atribuir uma categoria
3. Adicionar uma descrição
4. Definir um nível de confiança

In [ ]:
# Exemplo de categorização
# Ajuste as categorias conforme sua análise visual

# Cluster 0
categorizer.categorize_cluster(
    cluster_id=0,
    category='vegetacao_densa',
    description='Área com vegetação densa, possivelmente floresta',
    confidence=0.9
)

# Cluster 1
categorizer.categorize_cluster(
    cluster_id=1,
    category='area_urbana',
    description='Região urbanizada com construções',
    confidence=0.85
)

# Continue para os demais clusters...

# Visualizar resultados da categorização
categorizer.plot_cluster_samples(X_samples, predictions)

## 4. Análise das Categorias

Vamos ver um resumo das categorias atribuídas:

In [ ]:
# Exibir resumo das categorias
summary = categorizer.get_category_summary()
display(summary)

## 5. Salvar Categorização

Vamos salvar as categorias para uso futuro:

In [ ]:
# Salvar categorias
categorizer.save_categories('../data/cluster_categories.json')

# Exemplo de como carregar posteriormente
loaded_categorizer = ClusterCategorizer.load_categories('../data/cluster_categories.json')

## 6. Aplicar Categorias em Novas Predições

Agora podemos usar as categorias para interpretar novas predições:

In [ ]:
# Fazer novas predições
new_predictions = model.predict(X_samples)

# Converter predições em categorias
categories = categorizer.predict_categories(new_predictions)

# Visualizar algumas predições com suas categorias
plt.figure(figsize=(15, 5))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(X_samples[i])
    plt.title(f'Categoria:\n{categories[i]}')
    plt.axis('off')
plt.tight_layout()

## 7. Dicas para Categorização

Ao categorizar os clusters, considere:

1. **Características Espectrais**:
   - Vegetação saudável tem alta reflectância no NIR (B8)
   - Água absorve fortemente no SWIR (B11)
   - Áreas urbanas têm alta reflectância no vermelho (B4)

2. **Padrões Espaciais**:
   - Áreas urbanas têm padrões geométricos
   - Florestas têm textura mais rugosa
   - Agricultura tem padrões regulares

3. **Nível de Confiança**:
   - Use valores mais altos (>0.8) quando a categoria é clara
   - Use valores mais baixos (<0.7) quando houver ambiguidade
   - Documente incertezas nas descrições

4. **Validação Cruzada**:
   - Compare com mapas de referência
   - Consulte especialistas quando possível
   - Valide com conhecimento local da área